In [17]:
import lightgbm as lgb
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import accuracy_score
seed = 42
import math

import seaborn as sns # statistical data visualization

In [18]:
def escribir_respuesta(ids,predicciones):
    with open("respuesta.csv",'w') as archivo:
        archivo.write("id,target\n")
        for i in range(len(ids)):
            linea = f"{int(ids[i])},{predicciones[i]}"
            archivo.write(f"{linea}\n")

In [19]:
train = pd.read_csv("../datasets/xgb-train.csv")
test = pd.read_csv("../datasets/xgb-test.csv")

In [20]:
import re
train = train.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))
test = test.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))

In [21]:
target = "target"
features = list(train.columns)
features.remove(target)
features.remove("Opportunity_ID")

#VANILLA NO
# features.remove("delivery_delay")
# features.remove("opportunity_lifetime")
# features.remove("converted_taxable_amount")
# features.remove("last_modified_to_delivery")
# features.remove("currency_conversion_rate")
# features.remove("Occur")
# features.remove("delivery_window")
# features.remove("account_creation_to_created_opp")

#VANILLA SI
features.remove('Total_Taxable_Amount')
#features.remove('ASP_(converted)')
features.remove("ASP")
features.remove("Total_Amount")
features.remove("Delivery_Year")
# features.remove("Week_Day")

#CONSIDERO REMOVIBLES
# features.remove("created_blocknum")   #FECHA!!
features.remove("late_delivery_blocknum")
features.remove("early_delivery_blocknum")
features.remove("last_modified_blocknum")
features.remove("account_creation_blocknum") #OJO CCON HIPOTESIS DEL CLIENTE VIEEJO
print("Features totales: {}".format(len(features)))

Features totales: 116


In [26]:
X, y = train.loc[:,features], train.loc[:,target]
X_test_Opp = test.loc[:,"Opportunity_ID"]
X_test = test.loc[:,features]

#print("Set de entrenamiento (size) {}".format(X_train.shape))
#print("Set de testing (size) {}".format(X_test.shape))

In [27]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.3, random_state = 0)

In [53]:
clf = lgb.LGBMClassifier()
test = test.drop_duplicates("Opportunity_ID")
X_test = X_test.drop_duplicates("Opportunity_ID")
ID_submit = test["Opportunity_ID"]
ID_submit.shape

KeyError: Index(['Opportunity_ID'], dtype='object')

In [48]:
y_pred=clf.predict_proba(X_test)

NotFittedError: Estimator not fitted, call `fit` before exploiting the model.

In [41]:
y_pred_df = pd.DataFrame(y_pred)
y_pred_df = y_pred_df.drop_duplicates("Opportunity_ID")

KeyError: Index(['Opportunity_ID'], dtype='object')

In [52]:
clf.fit(X_train,y_train)
pred = clf.predict_proba(X_test)
pred_df = pd.DataFrame(pred, columns=["Other","Target"])
pred_df["Opportunity_ID"] = ID_submit
pred_df.drop(columns="Other",inplace=True)
pred_df

,Target,Opportunity_ID
0,0.964049,10689.0
1,0.959751,NaN
2,0.965264,NaN
3,0.704063,10690.0
4,0.828785,NaN
...,...,...
2546,0.988687,NaN
2547,0.055551,12365.0
2548,0.053610,12366.0
2549,0.558551,12367.0


In [ ]:
accuracy=accuracy_score(y_pred, y_test)
print('LightGBM Model accuracy score: {0:0.4f}'.format(accuracy_score(y_test, y_pred)))

In [ ]:
y_pred_train = clf.predict(X_train)
print('Training-set accuracy score: {0:0.4f}'. format(accuracy_score(y_train, y_pred_train)))

In [ ]:
print('Training set score: {:.4f}'.format(clf.score(X_train, y_train)))

print('Test set score: {:.4f}'.format(clf.score(X_test, y_test)))

In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print('Confusion matrix\n\n', cm)
print('\nTrue Positives(TP) = ', cm[0,0])
print('\nTrue Negatives(TN) = ', cm[1,1])
print('\nFalse Positives(FP) = ', cm[0,1])
print('\nFalse Negatives(FN) = ', cm[1,0])

In [ ]:
cm_matrix = pd.DataFrame(data=cm, columns=['Actual Positive:1', 'Actual Negative:0'], 
                                 index=['Predict Positive:1', 'Predict Negative:0'])

sns.heatmap(cm_matrix, annot=True, fmt='d', cmap='YlGnBu')

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

In [ ]:
params = {
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': 'binary_logloss',
    'max_depth': 8,
    'num_leaves': 100,
    'learning_rate': 0.1,
    'verbose': 0, 
    'early_stopping_round': 1000}
n_estimators=99999999

In [ ]:
d_train = lgb.Dataset(X_train.values, label=y_train.values)
d_valid = lgb.Dataset(X_val.values, label=y_val.values)
watchlist = [d_valid]
reg = lgb.train(params, d_train, n_estimators, watchlist, verbose_eval=500)

In [ ]:
Y_pred = reg.predict(X_val.values)

f = np.vectorize(math.exp)
Y_pred = f(Y_pred)
Y_val = f(y_val.values)
mean_absolute_error(Y_val,Y_pred)

In [ ]:

test_predict = reg.predict(X_test)
f = np.vectorize(math.exp)
test_predict = f(test_predict)
test_predict
#escribir_respuesta(ids, test_predict)